In [9]:
import os
import sys
import torch 
from torch import nn
from typing import Dict, Tuple
from torch.nn import Linear, LayerNorm, TransformerEncoder, TransformerEncoderLayer, ModuleList
import torch.nn.functional as F
from einops import rearrange
import itertools
import numpy as np
import copy
import scipy 
from scipy.io import loadmat
from scipy.signal import butter, filtfilt
import glob
import pandas as pd
import re
import shutil
import os
from typing import Sequence
import torch.nn.functional as F
from einops import rearrange
#from util.graph import Graph
import math
from torch._dynamo import explain
sys.path.append(os.path.dirname(os.getcwd()))
from Models.transformer import TransModel

## Preprocessing ##

In [8]:
def avg_pool(sequence, window_size = 5, stride=1, max_length = 512 , shape = None):
    shape = sequence.shape
    sequence = sequence.reshape(shape[0], -1)
    sequence = np.expand_dims(sequence, axis = 0).transpose(0,2, 1)
    sequence = torch.tensor(sequence, dtype=torch.float32)
    stride =  ((sequence.shape[2]//max_length)+1 if max_length < sequence.shape[2] else 1)
    sequence = F.avg_pool1d(sequence,kernel_size=window_size, stride=stride)
    sequence = sequence.squeeze(0).numpy().transpose(1,0)
    sequence = sequence.reshape(-1, *shape[1:])
    return sequence


def pad_sequence_numpy(sequence: np.ndarray, max_sequence_length: int, input_shape: Sequence[int]) -> np.ndarray:
    shape = list(input_shape)
    shape[0] = max_sequence_length
    pooled_sequence = avg_pool(sequence=sequence, max_length = max_sequence_length, 
                               shape = input_shape)
    new_sequence = np.zeros(shape, sequence.dtype)
    new_sequence[:len(pooled_sequence)] = pooled_sequence
    return new_sequence

  
def sliding_window(data, clearing_time_index, max_time, sub_window_size, stride_size):

    assert clearing_time_index >= sub_window_size - 1 , "Clearing value needs to be greater or equal to (window size - 1)"
    start = clearing_time_index - sub_window_size + 1 

    if max_time >= data.shape[0]-sub_window_size:
        max_time = max_time - sub_window_size + 1
        # 2510 // 100 - 1 25 #25999 1000 24000 = 24900

    sub_windows  = (
        start + 
        np.expand_dims(np.arange(sub_window_size), 0) + 
        np.expand_dims(np.arange(max_time, step = stride_size), 0).T
    )

    #labels = np.round(np.mean(labels[sub_windows], axis=1))
    return data[sub_windows] 

def process_data(raw_data, window_size, stride):
    # dataframe = pd.read_csv(file_path)
    # raw_data = dataframe[['w_accelerometer_x', 'w_accelerometer_y', 'w_accelerometer_z']].to_numpy()
    # labels = dataframe['outcome'].to_numpy()
    data = sliding_window(raw_data, window_size - 1,raw_data.shape[0],window_size,stride)
    return data

def select_subwindow_pandas(unimodal_data):
        n = len(unimodal_data)
        magnitude = np.linalg.norm(unimodal_data, axis = 1)
        df = pd.DataFrame({"values":magnitude})
        #250
        df["variance"] = df["values"].rolling(window=125).var()

        # Get index of highest variance
        max_idx = df["variance"].idxmax()

        # Get segment
        final_start = max(0, max_idx-100)
        final_end = min(n, max_idx + 100)
        return unimodal_data[final_start:final_end, :]

In [21]:
def sf_processing(data_dir = '/Users/tousif/LightHART/data/smartfallmm', subjects =[32,39,30,31,33,34,35,37,43,44,45,36,29],
                    skl_window_size = 32, 
                    num_windows = 10,
                    acc_window_size = 32,
                    num_joints = 32, num_channels = 3):
    count = 0 
    skl_set = []
    acc_set = []
    label_set = []
    file_paths = glob.glob(f'{os.path.dirname(os.getcwd())}/data/smartfallmm/young/skeleton/*.csv')
    print("file paths {}".format(len(file_paths)))

    acc_dir = f"{os.path.dirname(os.getcwd())}/data/smartfallmm/young/accelerometer/watch"
    pattern = r'S\d+A\d+T\d+'
    act_pattern = r'(A\d+)'
    label_pattern = r'(\d+)'

    for idx,path in enumerate(file_paths):
        desp = re.findall(pattern, path)[0]
        if not int(desp[1:3]) in subjects:
            continue
        act_label = re.findall(act_pattern, path)[0]
        label =int(int(re.findall(label_pattern, act_label)[0]) > 9)
        acc_path = f'{acc_dir}/{desp}.csv'



        try: 
            acc_df = pd.read_csv(acc_path, header = 0).dropna()
            acc_data = acc_df.bfill().iloc[2:, -3:].to_numpy(dtype=np.float32)
            
            skl_df  = pd.read_csv(path, index_col =False).dropna()
            skl_data = skl_df.bfill().iloc[:, -96:].to_numpy(dtype=np.float32)
            if acc_data.shape[0] > 250: 
                 acc_data = select_subwindow_pandas(acc_data)

            acc_window = process_data(acc_data, 128, 32)    
            skl_window = process_data(skl_data, 128, 32)
            #skl_data = rearrange(skl_window, 't (j c) -> t j c' , j = 32, c = 3)
            acc_set.append(acc_window)
            skl_set.append(skl_window)
            label_set.append(np.repeat(label, repeats=len(acc_window)))
        except Exception as e:
                print(e)
                count =+ 1
    concat_acc = np.concat(acc_set, axis = 0)
    concat_skl = np.concat(skl_set, axis = 0)
    # #s,w,j,c = concat_skl.shape
    concat_label = np.concat(label_set, axis = 0)
     #print(concat_acc.shape[0], concat_label.shape[0])
    # _,count  = np.unique(concat_label, return_counts = True)
    # print(concat_acc.shape)
    # print(concat_skl.shape)
    # #np.savez('/home/bgu9/KD_Multimodal/train.npz' , data = concat_acc, labels = concat_label)
    print(count)
    dataset = { 'accelerometer' : concat_acc,
                 'skeleton' : concat_skl, 
                 'labels': concat_label}
    return dataset

In [22]:
class UTD_mm(torch.utils.data.Dataset):
    def __init__(self, dataset, batch_size):
        self.inertial_modality = next((modality for modality in dataset if modality in ['accelerometer', 'gyroscope']), None)
        #self.acc_data = dataset[self.inertial_modality]
        self.acc_data = dataset['accelerometer']
        self.labels = dataset['labels']
        self.skl_data = dataset['skeleton']
        #self.skl_data = np.random.randn(self.acc_data.shape[0], 32,3)
        self.num_samples = self.acc_data.shape[0]
        self.acc_seq = self.acc_data.shape[1]
        self.skl_seq, self.skl_length, self.skl_features = self.skl_data.shape
        self.skl_data = self.skl_data.reshape(self.skl_seq, self.skl_length, -1, 3)
        self.channels = self.acc_data.shape[2]
        self.batch_size = batch_size
        self.transform = None
        self.crop_size = 64

    
    def random_crop(self,data : torch.Tensor) -> torch.Tensor:
        '''
        Function to add random cropping to the data
        Arg: 
            data : 
        Output: 
            crop_data: will return croped data
        '''
        length = data.shape[0]
        start_idx = np.random.randint(0, length-self.crop_size-1)
        return data[start_idx : start_idx+self.crop_size, :]

    def cal_smv(self, sample : torch.Tensor) -> torch.Tensor:
        '''
        Function to calculate SMV
        '''
        mean = torch.mean(sample, dim = -2, keepdim=True)
        zero_mean = sample - mean
        sum_squared =  torch.sum(torch.square(zero_mean), dim=-1, keepdim=True) 
        smv= torch.sqrt(sum_squared)
        return smv
    
    def calculate_weight(self, sample):
        """
        Calculate the magnitude (weight) of accelerometer data.

        Parameters:
        - data: A PyTorch tensor of shape (128, 3) where each row is [ax, ay, az].

        Returns:
        - A 1D PyTorch tensor of shape (128,) containing the magnitude for each row.
        """
        mean = torch.mean(sample, dim = -2, keepdim=True)
        zero_mean = sample - mean
        return torch.sqrt(torch.sum(zero_mean**2, dim=-1, keepdim=True))
    
    def calculate_pitch(self,data):
        """
        Calculate the pitch from accelerometer data.

        Parameters:
        - data: A PyTorch tensor of shape (128, 3) where each row is [ax, ay, az].

        Returns:
        - A 1D PyTorch tensor of shape (128,) containing the pitch angle for each row in radians.
        """
        ax = data[:, 0]
        ay = data[:, 1]
        az = data[:, 2]
        return torch.atan2(-ax, torch.sqrt(ay**2 + az**2)).unsqueeze(1)
    

    def calculate_roll(self,data):
        """
        Calculate the roll from accelerometer data.

        Parameters:
        - data: A PyTorch tensor of shape (128, 3) where each row is [ax, ay, az].

        Returns:
        - A 1D PyTorch tensor of shape (128,) containing the roll angle for each row in radians.
        """
        ax = data[:, 0]
        ay = data[:, 1]
        az = data[:, 2]
        return torch.atan2(ay, az).unsqueeze(1)




    
    def __len__(self):
        return self.num_samples

    def __getitem__(self, index):
        acc_data = torch.tensor(self.acc_data[index, : , :])
        data = dict()

        watch_smv = self.cal_smv(acc_data)
        watch_weight = self.calculate_weight(acc_data)
        # watch_roll = self.calculate_roll(acc_data)
        # watch_pitch = self.calculate_pitch(acc_data)
        acc_data = torch.cat(( watch_smv,acc_data), dim = -1)
        #data[self.inertial_modality] = acc_data

        data['accelerometer'] = acc_data
        label = self.labels[index]
        label = torch.tensor(label)
        label = label.long()
        return data, label, index



### Model Definition ##

In [23]:

class TransformerEncoderWAttention(nn.TransformerEncoder):
    def forward(self, src, mask = None, src_key_padding_mask = None):
        output = src
        for layer in self.layers :
            output, attn = layer.self_attn(output, output, output, attn_mask = mask,
                                            key_padding_mask = src_key_padding_mask, need_weights = True)
            output = layer(output, src_mask = mask, src_key_padding_mask = src_key_padding_mask)
        return output


class TransModel(nn.Module):
    def __init__(self,
                mocap_frames = 128,
                num_joints = 32,
                acc_frames = 128,
                num_classes:int = 8, 
                num_heads = 2, 
                acc_coords = 3, 
                av = False,
                num_layer = 2, norm_first = True, 
                embed_dim= 16, activation = 'relu',
                **kwargs) :
        super().__init__()

        ##### uncomment if want to test embedding network ##########
        # self.ts2vec_model = TSEncoder(input_dims = acc_coords, output_dims= 64)
        # self.embeding_model = torch.optim.swa_utils.AveragedModel(self.ts2vec_model)
        # self.embeding_model.load_state_dict(torch.load('/home/bgu9/LightHART/Models/model.pkl'))
        # self.freeze_pretrained()

        self.data_shape = (acc_frames, acc_coords)
        self.length = self.data_shape[0]
        size = self.data_shape[1]
        self.input_proj = nn.Sequential(nn.Conv1d(4, embed_dim, kernel_size=8, stride=1, padding='same'), 
                                        nn.BatchNorm1d(embed_dim))



        #dropout = 0.3 best
        self.encoder_layer = TransformerEncoderLayer(d_model = embed_dim,  activation = activation, 
                                                     dim_feedforward =embed_dim*2, nhead = num_heads,dropout=0.5)
        
        self.encoder = TransformerEncoderWAttention(encoder_layer = self.encoder_layer, num_layers = num_layer, 
                                          norm=nn.LayerNorm(embed_dim))

        self.output = nn.Linear(embed_dim, num_classes)
        self.temporal_norm = nn.LayerNorm(embed_dim)
        # self.ln1 = nn.Conv1d(64,32, kernel_size=3, stride=1, padding = 1)
        # self.bn1 = nn.BatchNorm1d(32)
        # self.drop1 = nn.Dropout(p = 0.5)
        # self.ln2 = nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=1)
        # self.bn2 = nn.BatchNorm1d(16)
        # self.drop2 = nn.Dropout(p = 0.5)
        
        # self.output = Linear(16, num_classes)
        #nn.init.normal_(self.output.weight, 0, math.sqrt(2. / num_classes))
    
    def freeze_pretrained(self):
         for params in self.embeding_model.parameters(): 
              params.requires_grad = False
    def unfreeze_pretrained(self, num_epochs): 
        layers = list(self.embeding_model.children())
        unfrozen_layers = layers[-200//num_epochs:]
        #  for params in self.embeding_model.parameters():
        #       params.requires_grad = True
        for layer in unfrozen_layers: 
             for param in layer.parameters():
                  param.requires_grad = True

    def forward(self, acc_data, **kwargs):
        epochs = kwargs.get('epoch')
        # if isinstance(epochs,int) and ( epochs+1) % 25 == 0: 
        #       self.unfreeze_pretrained(epochs)
        # b, l, c = acc_data.shape
        # x = rearrange(acc_data, 'b l c -> b c l')
        # x = self.embeding_model(acc_data)
       
        
        ###### for transformer ############
        x = rearrange(acc_data, 'b l c -> b c l')
        x = self.input_proj(x) # [ 8, 64, 3]
        x = rearrange(x, 'b c l -> l b c ')
        x = self.encoder(x)
        x = rearrange(x ,'l b c -> b l c')

        x = self.temporal_norm(x)
        #feature = rearrange(x, 'b l c -> b c l' )
        feature = x
        # feature = F.avg_pool1d(feature, kernel_size=feature.shape[-1], stride=1)
        # feature = torch.flatten(feature, 1)
        x = rearrange(x, 'b f c -> b c f')
        x = F.avg_pool1d(x, kernel_size = 128, stride = 1)
        x = rearrange(x, 'b c f -> b (c f)')
        x = F.sigmoid(self.output(x))
        return x 

In [24]:
data = torch.randn(size = (64,128,4))
skl_data = torch.randn(size = (64,128,32,3))
model = TransModel(num_layer=2, norm_first=True, embed_dim=32, activation='relu', acc_coords=3, num_classes=1, acc_frames=128, mocap_frames=128, num_heads=4)
model.load_state_dict(torch.load('/home/bgu9/LightHART/exps/smartfall_fall_wokd/student/watch_acc_wobwandrollingfall_not_distilled/ttfstudent_31.pth'))
model_for_test = copy.deepcopy(model)


/home/bgu9/local/envs/execu_env/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


## Model Convertion ##

In [25]:
import ai_edge_torch
edge_model = ai_edge_torch.convert(model.eval(), (data,))
edge_model.export('watch_kd_128.tflite')

INFO:tensorflow:Assets written to: /tmp/tmpqsp1g7ps/assets


INFO:tensorflow:Assets written to: /tmp/tmpqsp1g7ps/assets
W0000 00:00:1744957863.981121  455215 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1744957863.981152  455215 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-18 06:31:03.981469: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpqsp1g7ps
2025-04-18 06:31:03.982333: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-18 06:31:03.982341: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpqsp1g7ps
2025-04-18 06:31:03.987815: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-18 06:31:04.023304: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpqsp1g7ps
2025-04-18 06:31:04.037505: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 56044 

## Model Testing ##

In [26]:

dataset = sf_processing(subjects =[32,31,39,30,33,34,35,37,43,44,45,36,29])

file paths 751
[Errno 2] No such file or directory: '/home/bgu9/LightHART/data/smartfallmm/young/accelerometer/watch/S34A03T05.csv'
[Errno 2] No such file or directory: '/home/bgu9/LightHART/data/smartfallmm/young/accelerometer/watch/S36A08T04.csv'
[Errno 2] No such file or directory: '/home/bgu9/LightHART/data/smartfallmm/young/accelerometer/watch/S34A11T03.csv'
[Errno 2] No such file or directory: '/home/bgu9/LightHART/data/smartfallmm/young/accelerometer/watch/S34A11T04.csv'
[Errno 2] No such file or directory: '/home/bgu9/LightHART/data/smartfallmm/young/accelerometer/watch/S35A13T04.csv'
[Errno 2] No such file or directory: '/home/bgu9/LightHART/data/smartfallmm/young/accelerometer/watch/S36A08T02.csv'
[Errno 2] No such file or directory: '/home/bgu9/LightHART/data/smartfallmm/young/accelerometer/watch/S45A07T03.csv'
[Errno 2] No such file or directory: '/home/bgu9/LightHART/data/smartfallmm/young/accelerometer/watch/S37A07T03.csv'
[Errno 2] No such file or directory: '/home/bgu9/

In [27]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
THRESHOLD = 0.5

def cal_prediction(logits):
    if isinstance(logits , torch.Tensor):
        return (logits>THRESHOLD).int().squeeze(1)
    else:
        return np.squeeze((logits>THRESHOLD).astype(int), axis = 1)


def cal_metrics(targets, predictions): 
    targets = np.array(targets)
    predictions = np.array(predictions)
    f1 = f1_score(targets, predictions)
    precision = precision_score(targets, predictions)
    recall = recall_score(targets, predictions)
    auc_score = roc_auc_score(targets, predictions)
    accuracy = accuracy_score(targets, predictions)
    return accuracy*100, f1*100, recall*100, precision*100, auc_score*100


In [28]:
dataloader = torch.utils.data.DataLoader(dataset = UTD_mm(dataset, batch_size=64), batch_size = 64, drop_last=True,shuffle=True)
label_list = []
prediction_list = []
model.eval()
for batch_idx, (data, labels, idx) in enumerate(dataloader):
    output = model_for_test(data['accelerometer'])
    preds = cal_prediction(output)
    prediction_list.extend(preds.tolist())
    label_list.extend(labels.tolist())

accuracy, f1, recall, precision, auc_score = cal_metrics(label_list, prediction_list)

print("Accuracy: {:2f}\n Precision: {:2f}\n Recall: {:2f}\n F1 Score {:2f}\n".format(accuracy, precision, recall, f1))


Accuracy: 81.250000
 Precision: 71.014493
 Recall: 95.543175
 F1 Score 81.472684



In [30]:
import tensorflow as tf
import numpy as np
import os

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

interpreter = tf.lite.Interpreter(model_path="watch_kd_128.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

prediction_list = []
label_list = []

# Set input tensor to the interpreter
for batch_idx, (data, labels, idx) in enumerate(dataloader):
    numpy_data = data['accelerometer'].detach().cpu().numpy()
    numpy_labels = labels.detach().cpu().numpy()
    tf_data = tf.convert_to_tensor(numpy_data)
    tf_labels =   tf.convert_to_tensor(numpy_labels)
    interpreter.set_tensor(input_details[0]['index'], tf_data)

    # Run inference
    interpreter.invoke()
    # Get the output tensor and post-process the results (example)
    output = interpreter.get_tensor(output_details[0]['index'])
    preds = cal_prediction(output)
    prediction_list.extend(preds.tolist())
    label_list.extend(labels.tolist())
accuracy, f1, recall, precision, auc_score = cal_metrics(label_list, prediction_list)

print("Accuracy: {:2f}\n Precision: {:2f}\n Recall: {:2f}\n F1 Score {:2f}\n".format(accuracy, precision, recall, f1))


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Accuracy: 83.173077
 Precision: 74.157303
 Recall: 92.957746
 F1 Score 82.500000

